<a href="https://colab.research.google.com/github/foobar167/articles/blob/master/Machine_Learning/code_examples/callbacks_usage_in_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the newest nightly build of TensorFlow
!pip install tf-nightly-2.0-preview
#!pip install tensorflow-gpu==2.0.0-alpha0

import tensorflow as tf
print(tf.__version__)

    100% |████████████████████████████████| 86.8MB 433kB/s 
    100% |████████████████████████████████| 3.1MB 10.1MB/s 
    100% |████████████████████████████████| 430kB 20.8MB/s 
    100% |████████████████████████████████| 61kB 14.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/89/67/41/63cbf0f6ac0a6156588b9587be4db5565f8c6d8ccef98202fc
Successfully built wrapt
thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
  Found existing installation: wrapt 1.10.11
    Uninstalling wrapt-1.10.11:
      Successfully uninstalled wrapt-1.10.11
2.0.0-dev20190424


In [11]:
print(tf.__version__)

2.0.0-dev20190424


In [30]:
import tensorflow as tf

# Just train to 90% accuracy and stop.
class myCallback1(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.9):
            print('\nReached 90% accuracy, so cancelling training!')
            self.model.stop_training = True

# Not useful for wavelike training.
class myCallback2(tf.keras.callbacks.Callback):
    def __init__(self, *args, **kwargs):
        self.patience = 10
        self.curr_patience = 0
        self.curr_evaluate = [float('inf')]

    def on_epoch_end(self, epoch, logs={}):
        evaluate = model.evaluate(x_test, y_test)
        print(evaluate[-1], self.curr_patience, self.curr_evaluate[-1])
        if evaluate[-1] > self.curr_evaluate[-1]:
            self.curr_patience += 1
        self.curr_evaluate = evaluate
        if self.curr_patience >= self.patience:
            print('\nModel has started to overfit, so cancelling training')
            self.model.stop_training = True

# Use EarlyStopping callback to stop training before overfitting
myCallback3 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

callback = myCallback2()  # select callback

mnist = tf.keras.datasets.fashion_mnist  # get link on the MNIST dataset

(x_train, y_train), (x_test, y_test) = mnist.load_data()  # load data
x_train, x_test = x_train / 255.0, x_test / 255.0  # normalize the model

# Build model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit(x_train, y_train, epochs=100, callbacks=[callback])


Epoch 1/100
10000/10000 [==============================] - 1s 55us/sample - loss: 0.4003 - accuracy: 0.8540
0.854 0 inf
60000/60000 [==============================] - 7s 120us/sample - loss: 0.4751 - accuracy: 0.8299
Epoch 2/100
10000/10000 [==============================] - 0s 48us/sample - loss: 0.3681 - accuracy: 0.8694
0.8694 0 0.854
60000/60000 [==============================] - 7s 111us/sample - loss: 0.3589 - accuracy: 0.8673
Epoch 3/100
10000/10000 [==============================] - 0s 49us/sample - loss: 0.3480 - accuracy: 0.8783
0.8783 1 0.8694
60000/60000 [==============================] - 7s 111us/sample - loss: 0.3238 - accuracy: 0.8794
Epoch 4/100
10000/10000 [==============================] - 0s 47us/sample - loss: 0.3769 - accuracy: 0.8595
0.8595 2 0.8783
60000/60000 [==============================] - 7s 110us/sample - loss: 0.2984 - accuracy: 0.8896
Epoch 5/100
10000/10000 [==============================] - 0s 46us/sample - loss: 0.3502 - accuracy: 0.8724
0.8724 2 0.85

In [23]:
# Evaluate model. Old evaluation.
print(model.evaluate(x_test, y_test))

10000/10000 [==============================] - 0s 49us/sample - loss: 0.9551 - accuracy: 0.8911
[0.9551087798453868, 0.8911]


In [31]:
# Evaluate model. New evaluation.
print(model.evaluate(x_test, y_test))

10000/10000 [==============================] - 1s 51us/sample - loss: 0.3749 - accuracy: 0.8854
[0.37486174614429474, 0.8854]
